#Understanding Parallelization of Machine Learning Algorithms in Apache Spark™

##Data Preparation

The dataset used for this example is Bank marketing. Given a set of features about a customer can we predict whether the person will open a term deposit account.

Original Source: [UCI Machine Learning Repository 
Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing)
[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

## 0. Grab the example dataset

In [0]:
%sh wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip -O /tmp/bank.zip

In [0]:
%sh unzip -o /tmp/bank.zip -d /tmp/bank

In [0]:
%fs mkdirs /BankMarketing

In [0]:
%sh cp -rv /tmp/bank /dbfs/BankMarketing

##1. Read the data into a Spark dataframe and write as a Delta table

In [0]:
df = spark.read.format("csv")\
.option("path", "dbfs:/BankMarketing/bank/bank-full.csv")\
.option("inferSchema", "true")\
.option("header", "true")\
.option("delimiter", ";")\
.option("quote", '"')\
.load()

In [0]:
df.write.mode('Overwrite').format("delta").saveAsTable("bank_marketing")

####Attribute Information:

####Input variables:
bank client data:
```
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone') 
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric) 
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric) 
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```

In [0]:
display(table("bank_marketing"))

## 2. Feature Engineering in Spark

In [0]:
input_data = table("bank_marketing")

cols = input_data.columns

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StringIndexerModel, VectorAssembler

categoricalColumns = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will be run all at once later on.
  stages += [stringIndexer, encoder]

In [0]:
#numericCols = ["age", "balance", "duration", "campaign", "previous", "day"]
numericCols = ["age", "balance", "campaign", "previous", "day"]

assemblerInputs = numericCols + [c + "classVec" for c in categoricalColumns]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [0]:
labelIndexer = StringIndexer(inputCol="y", outputCol="label")

stages += [labelIndexer]

In [0]:
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(input_data)
dataset = pipelineModel.transform(input_data)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
display(dataset)

## 3. Write out the Training and Testing data for future reference

In [0]:
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], seed = 12)

In [0]:
trainingData.write.mode("overwrite").format("delta").save("dbfs:/ml-workshop-datasets/employee/delta/trainingData")
testData.write.mode("overwrite").format("delta").save("dbfs:/ml-workshop-datasets/employee/delta/testData")